In [173]:
import csv
import string
import time
import re

sell_words = []
special_words = ["back us", "back me", "help us", "help me", "raise", "hurry", "detail", "support",
                 "so close", "hours to go", "days to go", "perk", "hour to go", "discount", "countdown",
                 "day to go", "reward", 'rewards', "goal", "stretch goal", "donate", "unlock", "hours", "mins to go",
                 "commission", "left","backer", "miss out", "less than", 'minutes to go', 'fund us',
                 'remaining days', 'new backers', 'final goal', ' final hours', 'new goal unlocked', 'stretch goal',
                 'almost there', 'pledges', 'fundraising', 'little bit helps', 'little bit helps', 'contribute to',
                 'order your', 'pre order','featured reward', 'early bird', 'restock', 'deluxe', 'funding goal',
                 'final day', 'available on kickstarter', 'preorder', 'can you help', "fund", "link in bio",
                 'approved', 'visit our',  ]
detection = []
translator = str.maketrans('', '', string.punctuation)

class Campaign():
    def __init__(self, title, cid, post, url, source, postID):
        initial_dict = {'source': source, 'text': post, 'postID': postID}
        self._title = title
        self._cid = cid
        self._post = post
        self._url = url
        self._source = source
        self._postdict = {}
        self._listofdict = []
        self._listofdict.append(initial_dict)

    def addListOfDict(self,listofdict):
        self._listofdict = self._listofdict + listofdict

    def addListOfDict(self,listofdict):
        self._listofdict = listofdict

    def addPost(self,post,source, postID):
        tempdict = {}
        tempdict['text'] = post
        tempdict['source'] = source
        tempdict['postID'] = postID
        self._listofdict.append(tempdict)
        return self._listofdict

def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString) and ("$" in inputString \
                                                            or "k" in inputString or "%" in inputString) \
                                                        and ("http" not in inputString)


def openSmallDataFile():
    """['', 1: 'post_id', 'handle', 'postDate', 'postType', 5:'gen_postType',
    'twitter_comment', 'twitter_retweet', 'postUrl', 'platform', 10: 'message', 'cta_prediction', 
    'relevant_prediction', 'comments', 'shares', 15: 'likes', 'views', 17:'CampaignId', 'campaign_name', 
    'campaign_startDate', 'campaign_endDate', 'project_goal', 'project_currency', 'project_staffPick', 
    'sub_category', 'project_url', 'project_pledgeAmount', 'project_backerCount', 'Succeeded']"""
    
    listOfCampaigns = []
    answers = []
    posts = []

    with open('data_input.csv', encoding = 'ascii', errors = 'ignore') as csvfile:
        reader = csv.reader(csvfile)
        contents = [row for row in reader]
        csvfile.close()
    #tables = contents[0]

    prev_obj = None

    for cell in contents[1:]:

        cid = cell[1]
        title = cell[2]
        post = cell[6]
        url = cell[4]
        source = cell[5]
        postID = cell[3]
        answers.append(cell[])

        """cid = cell[17]
        title = cell[18]
        post = cell[10]
        url = cell[8]
        source = cell[9]
        postID = cell[1]"""

        if prev_obj == None:
            obj = Campaign(title, cid, post, url, source, postID)
            posts.append(postID)
            prev_obj = obj
        elif prev_obj._cid == cid:
            prev_obj.addPost(post, source, postID)
            posts.append(postID)
        else:
            listOfCampaigns.append(prev_obj)
            obj = Campaign(title, cid, post, url, source, postID)
            posts.append(postID)
            prev_obj = obj
    listOfCampaigns.append(prev_obj)
    return [listOfCampaigns, answers, posts]


def sorter(post_text):
    global special_words
    global sell_words
    post_text = str.lower(post_text)
    #post_text = re.split('\W+', post_text)
    for s_word in special_words:
        if s_word in post_text:
            detection.append("1")
            return "1"
    #post_text = post_text.translate(translator)

    for word in post_text.split():
        if hasNumbers(word):
            detection.append("1")
            return "1"

    # for word in copy_post_text.split():
    #     if "http" in word[:4] and "_" not in word:
    #         if len(word.partition("\\")[0]) < 25:
    #             print(word.partition("\\")[0])
    #             urls.append(word.partition("\\")[0])
    #
    # if len(urls) > 0:
    #     real_urls = runlinks(urls)
    #     for real_url in real_urls:
    #         if "kickstarter" in real_url:
    #             detection.append("1")
    #             return "1"

    detection.append("0")
    return "0"


def assign(posts_list):
    for post in posts_list:
        num = sorter(post['text'])
        post['sell'] = num
    return posts_list


def success_rate(answer_list, complete_post_list):
    f = open('export.csv', 'w')
    writer = csv.writer(f)
    writer.writerow( ('CategoryNames','CategoryNumbers') )
    for i,j in zip(detection, complete_post_list):
        f.write(str(i)+","+str(j)+"\n")
    f.close()
    """
    wrong = []
    right = 0
    wrongAnswers = []
    for det, ans, post in zip(detection, answer_list, complete_post_list):
        if det == ans:
            right += 1
        else:
            wrong.append(post)
            wrongAnswers.append(det)
            """
    #return [(right/len(detection) *100), wrong, wrongAnswers]


def main():
    output = openSmallDataFile() #Joe's code

    campaign_list = output[0]
    answer_list = output[1]
    complete_post_list = output[2]
    new_campaign_list = []

    start_time = time.time()

    for campaign in campaign_list:
        postsList = campaign._listofdict
        newPostsList = assign(postsList)
        campaign.posts = newPostsList
        new_campaign_list.append(campaign)

    output2 = success_rate(answer_list, complete_post_list)
    #outputcsv(output2[1], output2[2])

    elapsed_time = time.time() - start_time
    print("Elapsed time: " + str(elapsed_time))


if __name__ == '__main__':
    main()

Elapsed time: 5.036935806274414
